In [3]:
import requests
import uuid
import json
import time
import os
import random

# ПОЛУЧАЕМ ДАННЫЕ О СПИСКЕ КОЛЛЕКЦИЙ

In [6]:
addr = "http://62.76.72.55:57888"
#addr = "http://localhost:57888"
id = str(uuid.uuid4())
message = { "jsonrpc": "2.0", "id": id, "method": "get_courses_data", "params":{"mqtt_key": "234", "user": "2321", "type": "courses", "data_key": "","action": "get_data"}}
resp = requests.post(f"{addr}/api/run", json=message)
resp.json()

{'jsonrpc': '2.0',
 'result': {'courses': []},
 'id': '280d754e-05b7-420f-9b24-fe3636f33dc1'}

In [2]:
message = { "id": id }
message = { "jsonrpc": "2.0", "id": id, "method": "get_message_result", "params":{}}
resp = requests.post("http://localhost:57888/api/run", json=message)
resp.json()

NameError: name 'requests' is not defined

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ (АСИНХРОННЫЙ СПОСОБ)

In [8]:
id = str(uuid.uuid4())
message = { "jsonrpc": "2.0", "id": id, "method": "add_message", "params":{"mqtt_key": "234", "user": "2321", "type": "problems", "data_key": "test", "action": "get_data"}}
resp = requests.post("http://localhost:57888/api/run", json=message)
resp

<Response [200]>

In [9]:
message = { "id": id }
message = { "jsonrpc": "2.0", "id": id, "method": "get_message_result", "params":{}}
resp = requests.post("http://localhost:57888/api/run", json=message)
resp.json()

{'jsonrpc': '2.0',
 'result': {'result': {'get_courses_data_processor': {'problems': [{'1': ['1'],
      'rating': 1,
      'task': '1. Напишите программу для знакомства с человеком и приветствия его (пользователь вводит Имя и Фамилию в двух отдельных строках, программа выводит в новой строке приветствие, например, при вводе имени «Иван» и фамилии «Петров» программа выводит : «Здравствуйте, Иван Петров !»)'},
     {'2': ['1'],
      'rating': 3,
      'task': '2. Напишите программу, которая выбирает максимальное и минимальное из пяти введенных чисел.'},
     {'3': ['1'],
      'rating': 2,
      'task': '3. Напишите программу, которая находит среднее из пяти введенных чисел.'},
     {'4': ['1'],
      'rating': 2,
      'task': '4. Найти периметр и площадь прямоугольного треугольника, если даны длины его катетов a и b. Значения а и b вводятся с клавиатуры.'},
     {'5': ['1'],
      'rating': 3,
      'task': '5. Дана длина ребра куба. Найти площадь грани, площадь полной поверхности и 

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ (СИНХРОННЫЙ СПОСОБ)

In [8]:
#addr = "http://cluster.vstu.ru:57888"
addr = "http://localhost:57888"
id = str(uuid.uuid4())
message = { "jsonrpc": "2.0", "id": id, "method": "get_courses_data", "params": {"mqtt_key": "234", "user": "2321", "type": "problems", "data_key": "test", "action": "get_data"}}
resp = requests.post(f"{addr}/api/run", json=message)
resp.json()

{'jsonrpc': '2.0',
 'result': {'problems': []},
 'id': '8b57a2a1-f1fd-44cb-82a2-1c8d515c54ae'}

# УДАЛЕНИЕ КОЛЛЕКЦИИ

In [15]:
#addr = "http://cluster.vstu.ru:57888"
addr = "http://localhost:57888"
course = "test"
id = str(uuid.uuid4())
message = { "jsonrpc": "2.0", "id": id, "method": "clear_data", "params": {"mqtt_key": "234", "user": "2321", "type": "course", "data_key": course, "action": "clear_data"}}
resp = requests.post(f"{addr}/api/run", json=message)
resp.json()

{'jsonrpc': '2.0',
 'result': {'test': 'Droped!'},
 'id': '1b3b5430-62e3-45ca-86fd-6a72e6037581'}

# СОСТАВЛЕНИЕ ФАЙЛА С ЗАДАЧАМИ

In [6]:
path = "/Users/user/Temp/vstu-pycourse-tests"
tests = []
for number in range(1, 90):
    fn = f'{path}/tests/{number}/result.json'
    fn1 = f'{path}/tasks/{number}.py'
    if os.path.exists(fn):
        with open(fn, encoding='utf-8') as f:
            var = json.load(f)
        with open(fn1, encoding='utf-8') as f:
            txt = f.read()
            ind1 = txt.find("'''")
            ind2 = txt.find("'''", 10)
            txt = txt[ind1+3:ind2].replace("\n", " ")
        test = {"problem": number, "rating": random.choice([1,2,3]), "task": txt, "type": "equal", "variants": {"1": var}}
        tests.append(test)
with open(f"{path}/test_problems.json", "w", encoding='utf-8') as f:
    json.dump(tests, f, ensure_ascii=False)

# ДОБАВЛЕНИЕ ЗАДАЧ ИЗ ФАЙЛА

In [12]:
course = "digital_test"
#addr = "http://cluster.vstu.ru:57888"
addr = "http://62.76.72.55:57888"
fn = "tests2.json"
with open(fn, encoding='utf-8') as f:
    data = json.load(f)
l = []
for p in data:
    if p["problem"] not in l and len(l) > 0:
        continue
    for v in p["variants"]:
        print(f'Problem {p["problem"]}, variant {v}')
        id = str(uuid.uuid4())
        message = {
            "jsonrpc": "2.0",
            "id": id,
            "method": "add_message",
            "params": {
                "mqtt_key": 123,
                "user": "admin", 
                "type": p["type"],
                "rating": p["rating"],
                "course": course, 
                "problem": p["problem"], 
                "variant": v,
                "action": "add_problem",
                "task": p.get("task", ""),
                "tests": p["variants"][v]
            }
        }
        r = requests.post(f"{addr}/api/run", json=message)
        print(f"{r}")
        message = { "jsonrpc": "2.0", "id": id, "method": "get_message_result", "params":{}}
        while True:
            time.sleep(1)
            resp = requests.post(f"{addr}/api/run", json=message)
            if resp.status_code == 200 and 'result' in resp.text:
                print(f"{resp.json()}")
                break


Problem 125, variant 1
<Response [200]>
{'jsonrpc': '2.0', 'result': {'add_problem_processor': {'task': 'Введите слова в строке через пробел, выведите наибольшее и наименьшее по длине из введенных слов , не используя функции max() и min(). Пример: при вводе hello, his name is VorLeonid ожидается: VorLeonid is.', 'problem': 125, 'type': 'equal', 'rating': 2, 'variants': {'1': {'1': {'in': 'hello, his name is VorLeonid', 'out': 'VorLeonid is', 'score': 10}}, '2': {'1': {'in': 'hi hellooooooo', 'out': 'hellooooooo hi', 'score': 10}}}}}, 'id': '3e4d5f4f-5e9d-460e-954f-46fd14f4a8ee'}
Problem 125, variant 2
<Response [200]>
{'jsonrpc': '2.0', 'result': {'add_problem_processor': {'task': 'Введите слова в строке через пробел, выведите наибольшее и наименьшее по длине из введенных слов , не используя функции max() и min(). Пример: при вводе hello, his name is VorLeonid ожидается: VorLeonid is.', 'problem': 125, 'type': 'equal', 'rating': 2, 'variants': {'1': {'1': {'in': 'hello, his name is Vor

# ПОЛУЧАЕМ ДАМП БАЗЫ

In [20]:
id = str(uuid.uuid4())
message = { "jsonrpc": "2.0", "id": id, "method": "get_base_dump", "params": {"date": "2023-05-30", "processor_name": "equal_processor", "admin_key": "e86e915a-249f-4fdc-a48a-7c3f9a0b36ff"}}
resp = requests.post(
    "http://localhost:57888/api/run", json=message)
resp.json()

{'jsonrpc': '2.0', 'result': [], 'id': 'ad98a740-5716-4ac1-85ab-d5029f0ff458'}

# ДОБАВЛЯЕМ/ИЗМЕНЯЕМ ИНФОРМАЦИЮ ПОЛЬЗОВАТЕЛЯ

In [23]:
id = str(uuid.uuid4())
message = { "jsonrpc": "2.0", "id": id, "method": "add_user_info", "params": { "user_name": "123456", "data": {"var": [1,2,3]} }}
resp = requests.post(
    "http://localhost:57888/api/run", json=message)
resp.json()

{'jsonrpc': '2.0',
 'result': 'sucess',
 'id': 'ba8edef9-5d5e-4452-b083-f9e39a02c1dc'}

# ПОЛУЧАЕМ ИНФОРМАЦИЮ ПОЛЬЗОВАТЕЛЯ

In [13]:
id = str(uuid.uuid4())
message = { "jsonrpc": "2.0", "id": id, "method": "get_user_info", "params": { "user_name": "*" }}
resp = requests.post(
    "http://62.76.72.55:57888/api/run", json=message)
resp.json()

ConnectionError: HTTPConnectionPool(host='localhost', port=57888): Max retries exceeded with url: /api/run (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10e777dd0>: Failed to establish a new connection: [Errno 61] Connection refused'))